# Filter cells+positions+variants using thresholds

Saving as: 
1) NT pileup format for mgatk, similar to the input
2) Filtered af_by_cell matrix. This may not get used b/c we will call variants with mgatk later, but nice to have
3) Sparse matrices of Depth + allele frequency. These could be used for vireo. 

Create a function to convert between NT pileup and sparse matrices.

In [ ]:
def convert_df_to_sparse(df, in_fmt="wide", out_f=None):
    sparse_df = pd.DataFrame()
    if out_f is not None:
        sparse_df.to_csv(out_f, sep='\t')
    return`

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 


In [2]:
import os
from os.path import join
import pandas as pd
from src.config import RESULTS, FIGURES_DIR, ROOT_DIR
from src.calculate_AF_by_cell import calculate_af
#ref_fa = "/data2/genome/human_GRCh38/cellranger/refdata-cellranger-GRCh38-3.0.0/fasta/genome.fa"
#ref_fa="/data2/mito_lineage/BWA-Primers-MT/MT_genome/MT.fasta"
maxBP = 16571


import numpy as np

Project Directory: /data2/mito_lineage


In [3]:
from matplotlib_venn import venn2

import seaborn as sns
import matplotlib.pyplot as plt

from mplh import cluster_help

fig_utils from mplh
here


In [4]:
os.chdir(ROOT_DIR)

## Files setup

In [5]:
indir=join(RESULTS,"mttrace")
mapq = "mapq_0"
num_reads = 200
cellr="True"
in_prefix = f'2020_11_18'
pbmc_j = "PBMC_J"
pbmc_p = "PBMC_P"



pbmc_j_indir = join(indir, in_prefix, pbmc_j,mapq,f"cellr_{cellr}")
pbmc_p_indir = join(indir, in_prefix, pbmc_p,mapq,f"cellr_{cellr}")

# sc_coverage_pbmc_j_f = join(pbmc_j_indir,"scPileup_concat_200",pbmc_j+"_200_all.coverage.txt.gz")
# sc_coverage_pbmc_p_f = join(pbmc_p_indir,"scPileup_concat_200",pbmc_p + "_200_all.coverage.txt.gz")
coverage_pbmc_j_dir = join(indir, in_prefix, pbmc_j, mapq, f"{pbmc_j}_scPileup_{num_reads}")
coverage_pbmc_p_dir = join(indir, in_prefix, pbmc_p, mapq, f"{pbmc_p}_scPileup_{num_reads}")

concat_pbmc_j_dir = join(pbmc_j_indir,f"{pbmc_j}_{num_reads}")
concat_pbmc_p_dir = join(pbmc_p_indir,f"{pbmc_p}_{num_reads}")

In [6]:
min_cells=100
min_reads=10
topN=0
min_het_cells=100
het_thresh=0.02

In [7]:
from src.utils.parse_config import read_config_file
p = read_config_file("parameters/2020_11_18_Croker_mito.yaml")

mt_ref_fa = p["mt_ref_fa"]

In [8]:
dirname = f"minC{min_cells}_minR{min_reads}_topN{topN}_hetC{min_het_cells}_hetT{het_thresh}_cellr{cellr}"
outdir = join(indir, f"lineage_trace/{in_prefix}/cellr_{cellr}/report", dirname)
if not os.path.exists(outdir):
    os.makedirs(outdir)
outdir

'/data2/mito_lineage/data/processed/mttrace/lineage_trace/2020_11_18/cellr_True/report/minC100_minR10_topN0_hetC100_hetT0.02_cellrTrue'

### Create allele frequency matrix for both, but add metadata for each cell about which experiment it came from
### Cluster and look

### Additional requirements is to remove poor-quality variants of BQ- see if Erick has the code of the mixture model

In [9]:
print(mt_ref_fa)
print(maxBP)
print(coverage_pbmc_p_dir)
print(concat_pbmc_p_dir)

/data2/mito_lineage/data/external/GRCh38_MT_blacklist/chrM.fasta
16571
/data2/mito_lineage/data/processed/mttrace/2020_11_18/PBMC_P/mapq_0/PBMC_P_scPileup_200
/data2/mito_lineage/data/processed/mttrace/2020_11_18/PBMC_P/mapq_0/cellr_True/PBMC_P_200


In [10]:
pbmc_p_AF_by_cell, pbmc_p_BQ_by_cell, pbmc_p_af, pbmc_p_bq_df, pbmc_p_nt_df = calculate_af(coverage_pbmc_p_dir, concat_pbmc_p_dir, ref_fasta=mt_ref_fa, 
                                                                                           AF_F=None, maxBP=maxBP, topN=topN, min_cells=min_cells, min_reads=min_reads, 
                                                                                           min_het_cells=min_het_cells, het_thresh=het_thresh)

no alt count 1
topN 0
Number of positions to keep : 16569
Number of cells to keep : 330
G not found for ATTACTCAGTAGCAAT-1
coverage not found for GTAGACTCAATGATGA-1
T not found for AGGCGTCCACGCGCAT-1
coverage not found for ACTACGACATCACAGT-1
C not found for GGAGGATTCCGGAAAG-1
G not found for ATTACTCAGTAGCAAT-1
coverage not found for GTAGACTCAATGATGA-1
T not found for AGGCGTCCACGCGCAT-1
coverage not found for ACTACGACATCACAGT-1
C not found for GGAGGATTCCGGAAAG-1
Positions that pass het filter: 107


In [11]:
print(f"Number of positions with heteroplasmy fraction above 0: {(pbmc_p_AF_by_cell>0).any(axis=0).sum()}")

Number of positions with heteroplasmy fraction above 0: 107
